In [1]:
!nvidia-smi

Wed Jan 20 13:08:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd "drive/My Drive/porijen_pytorch/notebook"

Mounted at /content/drive
/content/drive/My Drive/porijen_pytorch/notebook


In [3]:
!pip install pip install reformer_pytorch

  Created wheel for axial-positional-embedding: filename=axial_positional_embedding-0.2.1-cp36-none-any.whl size=2904 sha256=97e8ed81a016b4a4482e5c1ebeab51811cdcdeb165333e1a8a723c93fcebd13d
  Stored in directory: /root/.cache/pip/wheels/cd/f8/93/25b60e319a481e8f324dcb1871aff818eb0c8143ed20b732b4
  Created wheel for product-key-memory: filename=product_key_memory-0.1.10-cp36-none-any.whl size=3072 sha256=da64a71b6e60184206b9ee58f80a96c728e0b0eca7aed4945127d48763c8ba3e
  Stored in directory: /root/.cache/pip/wheels/6d/e0/3b/fd3111a4fac652ed014ccfd4757754f006132723985e229419
Successfully built axial-positional-embedding product-key-memory


In [4]:
import os
import sys
import json
import glob
import torch

In [5]:
base_dir = os.path.dirname(os.getcwd())
out_dir = os.path.join(base_dir, "results", "models", "20210105")
data_dir = os.path.join(base_dir, "data", "preprocessed")
train_files = glob.glob(os.path.join(data_dir, "train", "*.json"))
valid_files = glob.glob(os.path.join(data_dir, "valid", "*.json"))
print(len(train_files), len(valid_files))

src_dir = os.path.join(base_dir, "src")
sys.path.append(os.path.join(src_dir))

5 5


In [6]:
from pytorch_trainer import Trainer, Reporter
from models import FacePolyGenConfig, FacePolyGen, FaceDataset, collate_fn_face

In [7]:
train_data = {
    "vertices": [],
    "faces": []
}
for file_path in train_files:
    with open(file_path) as fr:
        datas = json.load(fr)
        for data in datas:
            train_data["vertices"].append(torch.tensor(data["vertices"], dtype=torch.int32))
            train_data["faces"].append([torch.tensor(d, dtype=torch.int32) for d in data["faces"]])

valid_data = {
    "vertices": [],
    "faces": []
}
for file_path in valid_files:
    with open(file_path) as fr:
        datas = json.load(fr)
        for data in datas:
            valid_data["vertices"].append(torch.tensor(data["vertices"], dtype=torch.int32))
            valid_data["faces"].append([torch.tensor(d, dtype=torch.int32) for d in data["faces"]])

print(len(train_data["vertices"]), len(train_data["faces"]))
print(len(valid_data["vertices"]), len(valid_data["faces"]))

7003 7003
1088 1088


In [8]:
idx  = 0
train_data["vertices"][idx].shape, len(train_data["faces"][idx])

(torch.Size([252, 3]), 319)

In [9]:
train_dataset = FaceDataset(train_data["vertices"], train_data["faces"])
valid_dataset = FaceDataset(valid_data["vertices"], valid_data["faces"])
len(train_dataset), len(valid_dataset)

(7003, 1088)

In [10]:
config = FacePolyGenConfig(embed_dim=128, src__reformer__depth=9, tgt__reformer__depth=9)
model = FacePolyGen(config)
# for p in model.parameters():
#    p.register_hook(lambda grad: torch.clamp(grad, -1., 1.))
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

src__max_seq_len changed, because of lsh-attention's bucket_size
before: 2400 --> after: 2592 (with bucket_size: 48)
tgt__max_seq_len changed, because of lsh-attention's bucket_size
before: 5600 --> after: 5664 (with bucket_size: 48)


In [11]:
batch_size = 8
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True, collate_fn=collate_fn_face)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size, shuffle=False, collate_fn=collate_fn_face)
len(train_loader), len(valid_loader)

(876, 136)

In [12]:
epoch_num = 300
report_interval = 10
save_interval = 1
eval_interval = 1

reporter = Reporter(print_keys=['main/loss', 'main/perplexity', 'main/accuracy'])
trainer = Trainer(
    model, optimizer, [train_loader, valid_loader], gpu="gpu",
    reporter=reporter, stop_trigger=(epoch_num, 'epoch'),
    report_trigger=(report_interval, 'iteration'), save_trigger=(save_interval, 'epoch'),
    log_trigger=(save_interval, 'epoch'), eval_trigger=(eval_interval, 'epoch'),
    out_dir=out_dir, ckpt_path=os.path.join(out_dir, 'ckpt')
)

restart from 4 epoch.


In [ ]:
trainer.run()

epoch: 0	iteration: 0	main/loss: 2.35703	main/perplexity: 10.55956	main/accuracy: 0.36536
epoch: 0	iteration: 10	main/loss: 2.36802	main/perplexity: 10.93436	main/accuracy: 0.37405
epoch: 0	iteration: 20	main/loss: 2.30958	main/perplexity: 10.19387	main/accuracy: 0.38811
epoch: 0	iteration: 30	main/loss: 2.43996	main/perplexity: 11.63585	main/accuracy: 0.36124
epoch: 0	iteration: 40	main/loss: 2.34207	main/perplexity: 10.60350	main/accuracy: 0.37650
epoch: 0	iteration: 50	main/loss: 2.43274	main/perplexity: 11.53853	main/accuracy: 0.36725
epoch: 0	iteration: 60	main/loss: 2.36385	main/perplexity: 11.06020	main/accuracy: 0.37932
epoch: 0	iteration: 70	main/loss: 2.34202	main/perplexity: 10.53165	main/accuracy: 0.37814
epoch: 0	iteration: 80	main/loss: 2.41854	main/perplexity: 11.41528	main/accuracy: 0.36249
epoch: 0	iteration: 90	main/loss: 2.37962	main/perplexity: 11.20092	main/accuracy: 0.38114
epoch: 0	iteration: 100	main/loss: 2.32763	main/perplexity: 10.48187	main/accuracy: 0.37582